## Agent Tools

In [3]:
# Load environment variables
import os
from dotenv import load_dotenv

load_dotenv()

# Euri API Key
EURI_API_KEY = os.getenv("EURI_API_KEY")

In [5]:
# Initialize llm model
from euriai.langchain import create_chat_model

model = create_chat_model(
    api_key=EURI_API_KEY,
    model="gpt-4.1-nano",
    temperature=0.2
)
model

EuriaiChatModel(api_key='euri-bbab1558c330864f0704ddb28f62e188423e6d65766a561edc32e93f34d74d43', temperature=0.2)

In [7]:
response = model.invoke("What is artificial intelligence?")
print(response.content)

Artificial intelligence (AI) refers to the development of computer systems and software that can perform tasks typically requiring human intelligence. These tasks include learning from data (machine learning), understanding natural language (natural language processing), recognizing images or speech, reasoning, problem-solving, and decision-making. AI aims to create systems that can adapt, improve their performance over time, and sometimes operate autonomously, mimicking aspects of human cognition.


In [19]:
import ast, math
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.agents import create_react_agent,AgentExecutor
from langchain_core.runnables import RunnableLambda, RunnableWithMessageHistory
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory
from langchain_core.chat_history import InMemoryChatMessageHistory as ChatMessageHistory

### Tools
- Calculator Tool

In [14]:
@tool("calculator", return_direct=True)
def calculator(expression: str) -> str:
    """
    Evaluate a numeric math expression . supprts + , -,*,/,**, parentheses 
    and all kind of mathamatical functions
    """
    allowed_nodes =(
        ast.Expression,
        ast.UnaryOp,
        ast.unaryop,
        ast.BinOp,
        ast.operator,
        ast.Num,
        ast.Load,
        ast.pow,
        ast.FunctionDef,
        ast.Module,
        ast.Expr,
        ast.Call,
        ast.Name,
        ast.arguments,
        ast.args,
        ast.Constant
    )
    
    allowed_names = {k:v for k,v in vars(math).items() if not k.startswith("_")}
    allowed_names.update({"abs": abs, "round": round,"min": min,"max": max})
    node = ast.parse(expression, mode="eval")
    
    for n in ast.walk(node):
        if not isinstance(n, allowed_nodes):
            raise ValueError(f"Expression contains invalid node {type(n)}")
        if isinstance(n, ast.Name) and n.id not in allowed_names:
            raise ValueError(f"Expression contains invalid name {n.id}")
    code = compile(node, "<string>", "eval")
    return str(eval(code, {"__builtins__": {}}, allowed_names))

In [16]:
math_prompt = ChatPromptTemplate.from_messages([(
    "system","you are a precise math assistant , you can use thse tools:\n{tools}\n"
    "when you are going to use this tools follw these instruction exactly the same format:\n"
    "Questions :......\nthought ...\nAction by using one of the tools access that you have [{tool_names}]\n "
    "alwasy finish with final give me a numeric answer to the question"),
    ("human","questions: {input}\n{agent_scratchpad}")
])

In [23]:
tools_math = [calculator]
tools_math

[StructuredTool(name='calculator', description='Evaluate a numeric math expression . supprts + , -,*,/,**, parentheses \nand all kind of mathamatical functions', args_schema=<class 'langchain_core.utils.pydantic.calculator'>, return_direct=True, func=<function calculator at 0x0000027753D179C0>)]

In [24]:
math_agent = create_react_agent(
    llm=model,
    prompt=math_prompt,
    tools=tools_math
)
math_agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={'tools': 'calculator(expression: str) -> str - Evaluate a numeric math expression . supprts + , -,*,/,**, parentheses \nand all kind of mathamatical functions', 'tool_names': 'calculator'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['tool_names', 'tools'], input_types={}, partial_variables={}, template='you are a precise math assistant , you can use thse tools:\n{tools}\nwhen you are going to use this tools follw these instruction exactly the same format:\nQuestions :......\nthought ...\nAction by using one of the tools access that you have [{tool_names}]\n alwasy finish with final give me a numeric answer to the question'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'inpu

In [25]:
memory_math = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    input_key="input",
    output_key="output",
)

In [27]:
math_exector = AgentExecutor(
    agent=math_agent,
    tools=tools_math,
    memory=memory_math,
    verbose=True,
    max_iterations=2,
    handle_parsing_errors=(
        "you did not follow the instruction that i have given you to use the tools"
    ),
)

- KB Search

In [28]:
@tool("kb_search", return_direct=True)
def kb_search(query: str) -> str:
    """a mock function to search from a knowledge base"""
    knowledge_base = {
        "what is the capital of france": "The capital of France is Paris.",
        "who is the president of the united states": "The president of the United States is Joe Biden.",
        "what is the largest mammal": "The largest mammal is the blue whale.",
    }
    query = query.lower().strip("?")
    for key,answer in knowledge_base.items():
        if key in query:
            return answer
    return "I don't know the answer to that question."

In [29]:
tools_kb = [kb_search]
tools_kb

[StructuredTool(name='kb_search', description='a mock function to search from a knowledge base', args_schema=<class 'langchain_core.utils.pydantic.kb_search'>, return_direct=True, func=<function kb_search at 0x000002775536FCE0>)]

In [30]:
kb_prompt = ChatPromptTemplate.from_messages([
    ("system","you are a helpful assistant that can answer question based on your knowledge base and you can use the following tools:\n{tools}\n"
    "when you are going to use this tools follw these instruction exactly the same format:\n"
    "Questions :......\nthought ...\nAction by using one of the tools access that you have [{tool_names}]\n "
    "alwasy finish with final answer to the question that you are able to search from my knowledge base"),
    ("human","questions: {input}\n{agent_scratchpad}")
])

In [33]:
agent_kb = create_react_agent(
    llm=model,
    prompt=kb_prompt,
    tools=tools_kb
)

In [35]:
mem_kb = ConversationBufferWindowMemory(
    memory_key="chat_history",
    return_messages=True,
    input_key="input",
    output_key="output",
    k=5
)

In [36]:
kb_executor = AgentExecutor(
    agent=agent_kb,
    tools=tools_kb,
    memory=mem_kb,
    verbose=True,
    max_iterations=2,
    handle_parsing_errors=(
        "you did not follow the instruction that i have given you to use the kb  tools"
    ),
)

In [37]:
router_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a router. Read the user message and output exactly one token:\n"
     "- MATH: if it asks for calculations.\n"
     "- KB: otherwise (tech education, RAG, agents, general know-how).\n"
     "Output only MATH or KB."),
    ("human", "{input}")
])

In [39]:
router_chain = router_prompt | model |StrOutputParser()

In [40]:
def _dispatcher(inputs:dict):
    """it will take new inputs from human and it will also attach hitory from memory """
    user_msg = inputs["input"]
    choice = router_chain.invoke({"input": user_msg}).strip().upper()
    print(f"router choice : {choice}")
    if choice == "MATH" or choice == "Math" or choice == "math":
        return math_exector.invoke({"input": user_msg})
    elif choice == "KB" or choice == "Kb" or choice == "kb":
        return kb_executor.invoke({"input": user_msg})
    else:
        return "I can only answer math and kb related questions"

In [41]:
dispatcher = RunnableLambda(_dispatcher)
dispatcher

RunnableLambda(_dispatcher)

In [42]:
_sessions = {}

def _get_history(session_id:str):
    if session_id not in _sessions:
        _sessions[session_id] = ChatMessageHistory()
    return _sessions[session_id]

In [43]:
orchestrator = RunnableWithMessageHistory(
    runnable=dispatcher,
    get_session_history=_get_history,
    input_key="input",
    history_messages_key="history"
)

In [44]:
cfg = {"configurable":{"session_id":"user1"}}

In [45]:
print(orchestrator.invoke({"input":"what is the capital of france?"},config=cfg))

router choice : KB


> Entering new AgentExecutor chain...
Could not parse LLM output: `Thought: The question is asking for the capital city of France. This is general knowledge that I am aware of.

Final answer: The capital of France is Paris.`
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE you did not follow the instruction that i have given you to use the kb  toolsCould not parse LLM output: `Questions: what is the capital of france?  
thought ...  
Action by using one of the tools access that you have [kb_search]  
query: "capital of France"  
Final answer: The capital of France is Paris.`
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE you did not follow the instruction that i have given you to use the kb  tools

> Finished chain.
{'input': 'what is the capital of france?', 'chat_history': [], 'output': 'Agent stopped due to iteration limit or time limit.'}
